In [1]:
import pygrib

path = "./data/all-canada-2023.grib"

grbs = pygrib.open(path)

In [2]:
grbs.messages

43800

In [3]:
grb = grbs.read(1)[0]


In [4]:
grb.values.shape

(69, 341)

In [5]:
lats, lns = grb.latlons()

In [6]:
import numpy as np

In [8]:
import requests

def get_elevation(lat, lon):
    url = f"https://api.open-elevation.com/api/v1/lookup?locations={lat},{lon}"
    print(url)
    response = requests.get(url)
    data = response.json()
    elevation = data["results"][0]["elevation"]
    print(data)
    return elevation


# 49.2646	-123.1648
# Vancouver
lat = 49.2646
lon = -123.1648

# Ottawa
# lat = 45.421
# lon = -75.69
elevation = get_elevation(lat, lon)  # m

https://api.open-elevation.com/api/v1/lookup?locations=49.2646,-123.1648
{'results': [{'latitude': 49.2646, 'longitude': -123.1648, 'elevation': 21.0}]}


In [9]:

data = grb.values
lats, lons = grb.latlons()

# Find the nearest grid point
# (This is a simple approach; consider more efficient nearest-neighbor search algorithms for large datasets)
abslat = np.abs(lats-lat)
abslon = np.abs(lons-lon)
c = np.maximum(abslat, abslon)

# Get the index of the minimum value
idx = np.argmin(c)

idx

print(lats.flat[idx], lons.flat[idx], data.flat[idx])


49.25 -123.25 0.62384033203125


In [10]:
import math


def relative_humidity(drybulb_temp_kelvin, dewpoint_temp_kelvin):
  # August-Roche-Magnus formula
  #
  # https://bmcnoldy.earth.miami.edu/Humidity.html
  # https://en.wikipedia.org/wiki/Clausius%E2%80%93Clapeyron_relation#Meteorology_and_climatology
  # 100*(EXP((17.625*TD)/(243.04+TD))/EXP((17.625*T)/(243.04+T)))
  # (Temperatures from this formula are in celcius)
  t = drybulb_temp_kelvin - 273.15
  td = dewpoint_temp_kelvin - 273.15
  return 100 * math.exp((17.625 * td) / (243.04 + td)) / math.exp((17.625 * t) / (243.04 + t))

print(relative_humidity(273.15 + (90 - 32) / 1.8, 273.15 + (63 - 32) / 1.8))


40.783503702475876


In [12]:
from datetime import datetime
import pytz

def to_local_datetime(lat, lon, utc_datetime):
    response = requests.get(f"http://timezonefinder.michelfe.it/api/0_{lon}_{lat}")
    location = response.json()
    timezone_str = location["tz_name"]
    timezone = pytz.timezone(timezone_str)
    return utc_datetime.astimezone(timezone)


# to_local_datetime(lat, lon, datetime.fromisoformat("2023-01-01T12:00:00Z"))

In [14]:
"""
[(20230101, 0, '10 metre U wind component', -0.43768310546875),
 (20230101, 0, '10 metre V wind component', -1.0580596923828125),
 (20230101, 0, '2 metre dewpoint temperature', 275.86936950683594),
 (20230101, 0, '2 metre temperature', 275.9749450683594),
 (20230101, 0, 'Total cloud cover', 1.0),
"""

from datetime import datetime

wind_u_key = '10 metre U wind component'
wind_v_key = '10 metre V wind component'
dewpoint_temp_key = '2 metre dewpoint temperature'
temp_key = '2 metre temperature'
cloud_cover_key = 'Total cloud cover'

grbs.seek(0)
by_datetime = {}

for grb in grbs.read():
  year = str(grb.dataDate)[:4]
  month = str(grb.dataDate)[4:6]
  day = str(grb.dataDate)[6:]

  time = '{:04d}'.format(grb.dataTime)
  hour = time[:2]
  minute = time[2:]

  dt = '{}-{}-{}T{}:{}:00+00:00'.format(year, month, day, hour, minute)
  value = grb.values.flat[idx]
  by_datetime.setdefault(dt, {})[grb.name] = value


# Convert 10m elevation wind speed to 2m elevation wind speed
# https://github.com/jeroenterheerdt/HAsmartirrigation/commit/fe3bb45c1cd7e96f81b41f0662c721af3d38b9a9
wind_speed_10m_to_2m_ratio = 4.87 / math.log((67.8 * 10) - 5.42)

results = []

import pysolar

for dt, vs in by_datetime.items():
  dt_obj = datetime.fromisoformat(dt)

  solar_altitude = pysolar.solar.get_altitude(lat, lon, dt_obj, elevation)
  solar_radiation = pysolar.radiation.get_radiation_direct(dt_obj, solar_altitude) if solar_altitude > 0 else 0

  temp = vs[temp_key]
  dewpoint_temp = vs[dewpoint_temp_key]
  wind_u = vs[wind_u_key]
  wind_v = vs[wind_v_key]
  wind_speed_10m = math.sqrt(wind_u * wind_u + wind_v * wind_v)

  wind_speed_2m = wind_speed_10m * wind_speed_10m_to_2m_ratio

  rh = relative_humidity(temp, dewpoint_temp)

  cloud_cover = vs[cloud_cover_key]

  row = {
    "datetime": dt_obj.isoformat(),
    "outsideAirTempF": float("{:.1f}".format((temp - 273.15) * 1.8 + 32)),
    "relativeHumidityPercent": float("{:.1f}".format(rh)),
    "windSpeedMph": float("{:.1f}".format(wind_speed_2m * 2.237)),
    "cloudCoverPercent": float("{:.1f}".format(cloud_cover * 100.0)),
    "solarIrradiance": {
      "altitudeDegrees": float("{:.1f}".format(solar_altitude)),
      "wattsPerSquareMeter": float("{:.1f}".format(solar_radiation))
    }
  }
  results.append(row)

results


[{'datetime': '2023-01-01T00:00:00+00:00',
  'outsideAirTempF': 43.9,
  'relativeHumidityPercent': 90.2,
  'windSpeedMph': 2.5,
  'cloudCoverPercent': 14.1,
  'solarIrradiance': {'altitudeDegrees': 2.5, 'wattsPerSquareMeter': 49.8}},
 {'datetime': '2023-01-01T01:00:00+00:00',
  'outsideAirTempF': 43.8,
  'relativeHumidityPercent': 90.8,
  'windSpeedMph': 2.5,
  'cloudCoverPercent': 21.8,
  'solarIrradiance': {'altitudeDegrees': -5.8, 'wattsPerSquareMeter': 0.0}},
 {'datetime': '2023-01-01T02:00:00+00:00',
  'outsideAirTempF': 43.5,
  'relativeHumidityPercent': 91.2,
  'windSpeedMph': 2.4,
  'cloudCoverPercent': 33.3,
  'solarIrradiance': {'altitudeDegrees': -14.8, 'wattsPerSquareMeter': 0.0}},
 {'datetime': '2023-01-01T03:00:00+00:00',
  'outsideAirTempF': 43.0,
  'relativeHumidityPercent': 91.8,
  'windSpeedMph': 2.8,
  'cloudCoverPercent': 48.1,
  'solarIrradiance': {'altitudeDegrees': -24.3, 'wattsPerSquareMeter': 0.0}},
 {'datetime': '2023-01-01T04:00:00+00:00',
  'outsideAirTempF'

In [15]:
import json

with open('results.json', 'w') as file:
    json.dump(results, file)